### Emotion Recognition

#### Data preparation

In [1]:
import numpy as np
import pandas as pd

In [2]:
# read the csv data into pandas dataframe
file_path = 'fer2013.csv'
df = pd.read_csv(file_path, dtype='a')

# take a look of dataframe
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [3]:
# check number of samples and emotions in dataframe
print('Number of samples in dataset: ', df.shape[0])
print('Emotions in dataset: ', np.unique(df['emotion']))

Number of samples in dataset:  35887
Emotions in dataset:  ['0' '1' '2' '3' '4' '5' '6']


Therer are 7 different pre-defined emotions in the data set.  
0: Angry 1: Disgust 2: Fear 3: Happy 4: Sad 5: Surprise 6: Neutral

In [4]:
# convert dataframe into x(facial image), y(label) data

# read label data as numpy array 
label = np.array(df['emotion'])
# read facial image as numpy array
img = np.array(df['pixels'])

# convert pixels to 2d (48, 48) image data
img_2d = []
for pixels in img:
      img_2d.append(np.array([int(x) for x in pixels.split()]).reshape(48, 48))

# convert to numpy array
img_2d = np.array(img_2d)

We need to expend image dimension to (48,48,1) and convert y label data to categorical data (one hot coding style) for later keras model input

In [5]:
# expend image dimension to 
img_2d = np.expand_dims(img_2d, -1)

# convert (0-6) label into (0-1) categorical data
from keras.utils import to_categorical
y = to_categorical(label)

Using TensorFlow backend.


Split dataset into trainning set and testing set (25%)

In [6]:
from sklearn.model_selection import train_test_split
Xtr, Xts, ytr, yts = train_test_split(img_2d, y, test_size=0.25, random_state=33)
print('Number of sample in training: ', Xtr.shape[0]) 
print('Number of sample in testing: ', Xts.shape[0])

Number of sample in training:  26915
Number of sample in testing:  8972


Normanilze image data to make sure each pixel value is (0-1)

In [7]:
Xtr = Xtr / 255
Xts = Xts / 255

#### Build Keras CNN model

In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import optimizers

In [9]:
# input shape for the model
input_shape = img_2d.shape[1:]

# create a sequential convolutional network
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)))
model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))
    
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
    
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
    
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
    
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Compile the model. For multicalss data, use 'categorical_crossentropy' as loss function and 'accuracy' as metrics.

In [10]:
# adam optimizer with learning_rate of 0.001
learning_rate = 0.001
adam = optimizers.Adam(lr = learning_rate)

# compile the model
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

Fit model with training data and validate with testing data

In [11]:
# bacth size and number of epochs
batch_size = 64
nepochs = 100

# fit the model
model.fit(Xtr, ytr, epochs=100, batch_size = batch_size,
          validation_data=(Xts, yts), shuffle = True)

Instructions for updating:
Use tf.cast instead.
Train on 26915 samples, validate on 8972 samples
Epoch 1/100
26915/26915 [==============================] - 80s 3ms/step - loss: 2.0986 - acc: 0.1995 - val_loss: 1.8721 - val_acc: 0.2478
Epoch 2/100
26915/26915 [==============================] - 74s 3ms/step - loss: 1.8619 - acc: 0.2362 - val_loss: 1.8361 - val_acc: 0.2478
Epoch 3/100
26915/26915 [==============================] - 74s 3ms/step - loss: 1.8392 - acc: 0.2491 - val_loss: 1.8275 - val_acc: 0.2478
Epoch 4/100
26915/26915 [==============================] - 75s 3ms/step - loss: 1.8292 - acc: 0.2496 - val_loss: 1.8202 - val_acc: 0.2478
Epoch 5/100
26915/26915 [==============================] - 76s 3ms/step - loss: 1.8213 - acc: 0.2497 - val_loss: 1.8189 - val_acc: 0.2478
Epoch 6/100
26915/26915 [==============================] - 76s 3ms/step - loss: 1.8153 - acc: 0.2514 - val_loss: 1.8154 - val_acc: 0.2478
Epoch 7/100
26915/26915 [==============================] - 76s 3ms/step - l

Epoch 58/100
26915/26915 [==============================] - 76s 3ms/step - loss: 0.7324 - acc: 0.7483 - val_loss: 1.2286 - val_acc: 0.6110
Epoch 59/100
26915/26915 [==============================] - 76s 3ms/step - loss: 0.7257 - acc: 0.7489 - val_loss: 1.3099 - val_acc: 0.6204
Epoch 60/100
26915/26915 [==============================] - 76s 3ms/step - loss: 0.7254 - acc: 0.7491 - val_loss: 1.2449 - val_acc: 0.5955
Epoch 61/100
26915/26915 [==============================] - 76s 3ms/step - loss: 0.7031 - acc: 0.7572 - val_loss: 1.3115 - val_acc: 0.6169
Epoch 62/100
26915/26915 [==============================] - 76s 3ms/step - loss: 0.6935 - acc: 0.7621 - val_loss: 1.2694 - val_acc: 0.6197
Epoch 63/100
26915/26915 [==============================] - 76s 3ms/step - loss: 0.6761 - acc: 0.7695 - val_loss: 1.3299 - val_acc: 0.6083
Epoch 64/100
26915/26915 [==============================] - 76s 3ms/step - loss: 0.6784 - acc: 0.7678 - val_loss: 1.3513 - val_acc: 0.5960
Epoch 65/100
26915/26915 [=

In [12]:
# save trained model to a file
model_json = model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("emotion_model.h5")